# Structured data learning with TabTransformer

In [1]:
# pip install -U tensorflow-addons

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from collections import Counter

2022-12-15 20:17:46.269095: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/local/apps/gcc/10.2.0/lib:/cm/local/apps/gcc/10.2.0/lib64:/cm/local/apps/gcc/10.2.0/lib32:/cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2022-12-15 20:17:46.269117: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/o/oananbeh/.conda/envs/notebook-env/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.8.0 and strictly below 2.11.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.0 and is not supported. 
Some things might work, some things might not

## Prepare the data



In [4]:
NUMERIC_FEATURE_NAMES=['AGE', 'PEFF_Temperature', 'PEFF_Respiratory_Rate', 'PEFF_Pulse',
       'PEFF_BP_Systolic', 'PEFF_BP_Diastolic', 'PEFF_Glasgow',
       'ABGF_Ph', 'ABGF_Pa_O2', 'ABGF_Pa_CO2','ABGF_HCO3',
       'WBC', 'PNN', 'Lymphocytes', 'Hemoglobin', 'Platelets',
       'Creatinine', 'ALT', 'LDH', 'FERRITIN', 'D_DIMER', 'CRP',
       'PROCALCITONI', 'TROPONIN', 'Pro_BNP', 'PTT', 'Vitamin_D', 'IL6',
       ]

In [5]:
len(NUMERIC_FEATURE_NAMES)

28

In [6]:
CATEGORICAL_FEATURES=['PatientId','G.Nationality','Gender','C_Diabetes', 'C_HTN',
       'C.Heart ischemic', 'C.Heart failure', 'C.Cardiomyopathies',
       'C.End stage renal', 'C.Hemodialysis', 'C.COPD',
       'C.Lung Interstitial Disease ', 'C.Bronchial Asthma',
       'C.Cerebrovascular', 'C.Neurologic (dementia)',
       'C.History of psychiatric', 'C.Cirrhosis', 'C.liver disease',
       'C.Obesity', 'C.Sick cell', 'C.Cancer', 'C.Solid organ transplant',
       'IS.Hematopoietic cell transplant', 'IS.HIV', 'IS.corticosteroids',
       'IS.Other immunosuppressing agents', 'IS.Other immunodeficiencies',
       'IS.Pregnancy', 'IS.Smoker',  'CSA_Fever', 'CSA_SOB', 'CSA.Chest pain', 'CSA.Confusion',
       'CSA.Hemoptysis', 'CSA.Diarrhea', 'CSA_Cough', 'CSA.Myalgia',
       'CSA.Headache', 'CSA.Abdominal Pain', 'CSA.Nausea or Vomiting',
       'CSA.Loss of Smell or Tast', 'PO.Condition', 'PO.Current Condition',
       'PEFF.Nasal Cannula', 'PEFF.Mask', 'PEFF.HFNC',
       'PEFF.If patient need prone position', 'PEFF.If patient intubated',
       'PEFF.If Patient required Psychiatric Consultation',
       'PEFF.Presence of thrombo-embolic', 'PEFF.Confirmed DVT',
       'PEFF.Confirmed Pulmonary embolism',
       'PEFF.Confirmed Myocardial infarction', 'PEFF.Confirmed CVA/TIA',
       'MPA_Antibiotics','Anticoagulant','Immunomodulators','antiviral',
       'XrayResult']  

In [7]:
len(CATEGORICAL_FEATURES)

59

In [8]:
# fg=(NUMERIC_FEATURE_NAMES+CATEGORICAL_FEATURES)

In [9]:
path='/home/o/oananbeh/notebook/Experment2/src/TabTransformer-multiclass_died/'
dataSetPath=path+'SMOT_died_DataSet.csv'

In [10]:
data=pd.read_csv(dataSetPath)

In [11]:
data.shape

(76, 92)

In [12]:
# dd=c+list(data.columns)

In [13]:
# newdata=pd.DataFrame()
# data['AGE']=data['AGE'].astype('int64')
# newdata['AGE']=data['AGE']


In [14]:
# for index, c in newdata.iterrows():
#     if (c['AGE']>=56 and c['AGE'] <=73):
#         data[['AGE']]=data[['AGE']].replace(c['AGE'], 'Baby Boomers')  
#     if (c['AGE']>=38 and c['AGE'] <=55):
#         data[['AGE']]=data[['AGE']].replace(c['AGE'], 'Generation-X')  
#     if (c['AGE'] <=37):
#         data[['AGE']]=data[['AGE']].replace(c['AGE'], 'Generation-Y')  
#     if (c['AGE']>=74):
#         data[['AGE']]=data[['AGE']].replace(c['AGE'], 'Silent Generation')  
  

In [15]:
for feature in CATEGORICAL_FEATURES[2:]:
    data[feature] = data[feature].map({1:'yes', 0:'no'})

In [16]:
data[CATEGORICAL_FEATURES]=data[CATEGORICAL_FEATURES].astype('object')

In [17]:
data.shape

(76, 92)

In [18]:
CSV_HEADER=list(data.columns)

In [19]:
dataSetPath=path+'SMOT_died_DataSet2.csv'

In [20]:
data.to_csv(dataSetPath, index=False, header=False)

In [21]:
final_dataSet = data.drop('OUTCOME', axis=1)
target = data.OUTCOME

In [22]:
# Run ML On SMOTE Dataset
#Create final train and test sets startified according to the target variable. 
X_train, X_test, y_train, y_test = train_test_split(final_dataSet,target, test_size = 0.30,stratify=target, 
                                                    random_state=123)

In [23]:
train_data=pd.concat([X_train, y_train],axis=1)
test_data=pd.concat([X_test, y_test],axis=1)

In [24]:
train_data = pd.read_csv(dataSetPath, header=None, names=CSV_HEADER)
test_data = pd.read_csv(dataSetPath, header=None, names=CSV_HEADER)
train_data=train_data.sample(frac = 0.7)
test_data=test_data.sample(frac = 0.3)
print(f"Train dataset shape: {train_data.shape}")
print(f"Test dataset shape: {test_data.shape}")

Train dataset shape: (53, 92)
Test dataset shape: (23, 92)


In [25]:
target = train_data.OUTCOME
print(f"train data class counts: {Counter(target)}")

train data class counts: Counter({'died=>3': 28, 'died=<3': 25})


In [26]:
target = test_data.OUTCOME
print(f"test data class counts: {Counter(target)}")

test data class counts: Counter({'died=<3': 12, 'died=>3': 11})


*Now* we store the training and test data in separate CSV files.

In [27]:
train_data_file = path+"train_data.csv"
test_data_file = path+"test_data.csv"

train_data.to_csv(train_data_file, index=False, header=False)
test_data.to_csv(test_data_file, index=False, header=False)

## Define dataset metadata

Here, we define the metadata of the dataset that will be useful for reading and parsing
the data into input features, and encoding the input features with respect to their types.

In [28]:

# A dictionary of the categorical features and their vocabulary.
CATEGORICAL_FEATURES_WITH_VOCABULARY={}
for featrue in CATEGORICAL_FEATURES:
    CATEGORICAL_FEATURES_WITH_VOCABULARY[featrue]=sorted(list(data[featrue].unique()))

# A list of the categorical feature names.
CATEGORICAL_FEATURE_NAMES = list(CATEGORICAL_FEATURES_WITH_VOCABULARY.keys())
# A list of all the input features.
FEATURE_NAMES = NUMERIC_FEATURE_NAMES + CATEGORICAL_FEATURE_NAMES
# A list of column default values for each feature.
COLUMN_DEFAULTS = [
    [0.0] if feature_name in NUMERIC_FEATURE_NAMES  else ["NA"]
    for feature_name in CSV_HEADER
]

# The name of the target feature.
TARGET_FEATURE_NAME = 'OUTCOME'
# A list of the labels of the target features.
# TARGET_LABELS = [' discharge=<2',' discharge=<4','died=<3',' discharge>4','died=>3','discharge=<1',' discharge=<3']
TARGET_LABELS = ['died=<3', 'died=>3']

## Configure the hyperparameters

The hyperparameters includes model architecture and training configurations.

In [29]:
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0001
DROPOUT_RATE = 0.2
BATCH_SIZE = 8
NUM_EPOCHS = 15

NUM_TRANSFORMER_BLOCKS = 3  # Number of transformer blocks.
NUM_HEADS = 4  # Number of attention heads.
EMBEDDING_DIMS = 16  # Embedding dimensions of the categorical features.
MLP_HIDDEN_UNITS_FACTORS = [
    2,
    1,
]  # MLP hidden layer units, as factors of the number of inputs.
NUM_MLP_BLOCKS = 2  # Number of MLP blocks in the baseline model.

## Implement data reading pipeline

We define an input function that reads and parses the file, then converts features
and labels into a[`tf.data.Dataset`](https://www.tensorflow.org/guide/datasets)
for training or evaluation.

In [30]:
target_label_lookup = layers.StringLookup(
    vocabulary=TARGET_LABELS, mask_token=None, num_oov_indices=0
)


def prepare_example(features, target):
    # print(target)
    target_index = target_label_lookup(target)
    # weights = features.pop(WEIGHT_COLUMN_NAME), weights
    return features, target_index 

# func_labels = lambda x, y: x, np.array(targets[y])


def get_dataset_from_csv(csv_file_path, batch_size=128, shuffle=False):
    dataset = tf.data.experimental.make_csv_dataset(
        csv_file_path,
        batch_size=batch_size,
        column_names=CSV_HEADER,
        column_defaults=COLUMN_DEFAULTS,
        label_name=TARGET_FEATURE_NAME,
        num_epochs=1,
        header=False,
        na_value="?",
        shuffle=shuffle,
    ).map(prepare_example, num_parallel_calls=tf.data.AUTOTUNE, deterministic=False) 
    # dataset.take(1)
    
    # df = pd.read_csv(csv_file_path,  names=CSV_HEADER)
    # # print(df.columns)
    # features = df.drop('OUTCOME', axis=1)
    # labels = df.pop('OUTCOME')
    # # print(labels.unique())
    # labels = labels.map(targets)
    # labels
    return dataset

2022-12-15 20:17:58.871389: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/local/apps/gcc/10.2.0/lib:/cm/local/apps/gcc/10.2.0/lib64:/cm/local/apps/gcc/10.2.0/lib32:/cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2022-12-15 20:17:58.871506: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/local/apps/gcc/10.2.0/lib:/cm/local/apps/gcc/10.2.0/lib64:/cm/local/apps/gcc/10.2.0/lib32:/cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2022-12-15 20:17:58.871558: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt

In [31]:
from keras import backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall_keras = true_positives / (possible_positives + K.epsilon())
    return recall_keras
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision_keras = true_positives / (predicted_positives + K.epsilon())
    return precision_keras 
def negative_predictive_value(y_true, y_pred):
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fn = K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))
    return tn / (tn + fn + K.epsilon())

In [32]:
class CategoricalTruePositives(keras.metrics.Metric):
    def __init__(self, name="categorical_true_positives", **kwargs):
        super(CategoricalTruePositives, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name="ctp", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.reshape(tf.argmax(y_pred, axis=1), shape=(-1, 1))
        values = tf.cast(y_true, "int32") == tf.cast(y_pred, "int32")
        values = tf.cast(values, "float32")
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, "float32")
            values = tf.multiply(values, sample_weight)
        self.true_positives.assign_add(tf.reduce_sum(values))

    def result(self):
        return self.true_positives

    def reset_state(self):
        # The state of the metric will be reset at the start of each epoch.
        self.true_positives.assign(0.0)

## Implement a training and evaluation procedure

In [33]:

def run_experiment(
    model,
    train_data_file,
    test_data_file,
    num_epochs,
    learning_rate,
    weight_decay,
    batch_size,
):

    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(),#change this 
        metrics=[keras.metrics.SparseCategoricalAccuracy(name="accuracy"),CategoricalTruePositives(),get_f1,precision,recall], #change this one 
    )
    #keras.metrics.AUC(name='AUC_ROC'),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()
    train_dataset = get_dataset_from_csv(train_data_file, batch_size, shuffle=True)
    validation_dataset = get_dataset_from_csv(test_data_file, batch_size)
    # df_train=pd.read_csv(train_data_file)
    # df_test=pd.read_csv(test_data_file)

    # ohc_df_train = pd.get_dummies(df_train)
    # ohc_df_test = pd.get_dummies(df_test)
    # train_dataset = tf.keras.utils.to_categorical(ohc_df_train, num_classes=7)
    # # validation_dataset = tf.keras.utils.to_categorical(ohc_df_test, num_classes=7)
    # print(train_features.shape)
    # print(train_labels.shape)
    print("Start training the model...")
    history = model.fit(
        train_dataset, epochs=num_epochs, validation_data=validation_dataset
    )
    print("Model training finished")

    _, accuracy,tp,f1,prec,rec= model.evaluate(validation_dataset, verbose=0)
    
    # print(f"Validation f1: {round(f1 * 100, 2)}%")
    # print(f"Validation accuracy: {round(accuracy * 100, 2)}%")
    # print(f"Validation Precision: {round(prec * 100, 2)}%")
    # print(f"Validation Recall: {round(rec * 100, 2)}%")
    print(f"Validation f1: {round(f1,6)}")
    print(f"Validation accuracy: {round(accuracy,6)}")
    print(f"Validation Precision: {round(prec,6 )}")
    print(f"Validation Recall: {round(rec,6)}")
    # print(f"categorical true positives: {round(tp)}")
    y_pred = model.predict(validation_dataset)
    return history


## Create model inputs

Now, define the inputs for the models as a dictionary, where the key is the feature name,
and the value is a `keras.layers.Input` tensor with the corresponding feature shape
and data type.

In [34]:

def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        if feature_name in NUMERIC_FEATURE_NAMES:
            
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype=tf.float32
            )
        else:
            print(feature_name)
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype=tf.string
            )
    return inputs


## Encode features

The `encode_inputs` method returns `encoded_categorical_feature_list` and `numerical_feature_list`.
We encode the categorical features as embeddings, using a fixed `embedding_dims` for all the features,
regardless their vocabulary sizes. This is required for the Transformer model.

In [35]:

def encode_inputs(inputs, embedding_dims):

    encoded_categorical_feature_list = []
    numerical_feature_list = []

    for feature_name in inputs:
        if feature_name in CATEGORICAL_FEATURE_NAMES:

            # Get the vocabulary of the categorical feature.
            vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]

            # Create a lookup to convert string values to an integer indices.
            # Since we are not using a mask token nor expecting any out of vocabulary
            # (oov) token, we set mask_token to None and  num_oov_indices to 0.
            lookup = layers.StringLookup(
                vocabulary=vocabulary,
                mask_token=None,
                num_oov_indices=0,
                output_mode="int",
            )

            # Convert the string input values into integer indices.
            encoded_feature = lookup(inputs[feature_name])

            # Create an embedding layer with the specified dimensions.
            embedding = layers.Embedding(
                input_dim=len(vocabulary), output_dim=embedding_dims
            )

            # Convert the index values to embedding representations.
            encoded_categorical_feature = embedding(encoded_feature)
            encoded_categorical_feature_list.append(encoded_categorical_feature)

        else:

            # Use the numerical features as-is.
            numerical_feature = tf.expand_dims(inputs[feature_name], -1)
            numerical_feature_list.append(numerical_feature)

    return encoded_categorical_feature_list, numerical_feature_list


## Implement an MLP block

In [36]:

def create_mlp(hidden_units, dropout_rate, activation, normalization_layer, name=None):

    mlp_layers = []
    for units in hidden_units:
        mlp_layers.append(normalization_layer),
        mlp_layers.append(layers.Dense(units, activation=activation))
        mlp_layers.append(layers.Dropout(dropout_rate))

    return keras.Sequential(mlp_layers, name=name)


## Experiment 2: TabTransformer

The TabTransformer architecture works as follows:

1. All the categorical features are encoded as embeddings, using the same `embedding_dims`.
This means that each value in each categorical feature will have its own embedding vector.
2. A column embedding, one embedding vector for each categorical feature, is added (point-wise) to the categorical feature embedding.
3. The embedded categorical features are fed into a stack of Transformer blocks.
Each Transformer block consists of a multi-head self-attention layer followed by a feed-forward layer.
3. The outputs of the final Transformer layer, which are the *contextual embeddings* of the categorical features,
are concatenated with the input numerical features, and fed into a final MLP block.
4. A `softmax` classifer is applied at the end of the model.

The [paper](https://arxiv.org/abs/2012.06678) discusses both addition and concatenation of the column embedding in the
*Appendix: Experiment and Model Details* section.
The architecture of TabTransformer is shown below, as presented in the paper.

<img src="https://raw.githubusercontent.com/keras-team/keras-io/master/examples/structured_data/img/tabtransformer/tabtransformer.png" width="500"/>

In [37]:

def create_tabtransformer_classifier(
    num_transformer_blocks,
    num_heads,
    embedding_dims,
    mlp_hidden_units_factors,
    dropout_rate,
    use_column_embedding=False,
):

    # Create model inputs.
    inputs = create_model_inputs()
    # encode features.
    encoded_categorical_feature_list, numerical_feature_list = encode_inputs(
        inputs, embedding_dims
    )
    # Stack categorical feature embeddings for the Tansformer.
    encoded_categorical_features = tf.stack(encoded_categorical_feature_list, axis=1)
    # Concatenate numerical features.
    numerical_features = layers.concatenate(numerical_feature_list)

    # Add column embedding to categorical feature embeddings.
    if use_column_embedding:
        num_columns = encoded_categorical_features.shape[1]
        column_embedding = layers.Embedding(
            input_dim=num_columns, output_dim=embedding_dims
        )
        column_indices = tf.range(start=0, limit=num_columns, delta=1)
        encoded_categorical_features = encoded_categorical_features + column_embedding(
            column_indices
        )

    # Create multiple layers of the Transformer block.
    for block_idx in range(num_transformer_blocks):
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embedding_dims,
            dropout=dropout_rate,
            name=f"multihead_attention_{block_idx}",
        )(encoded_categorical_features, encoded_categorical_features)
        # Skip connection 1.
        x = layers.Add(name=f"skip_connection1_{block_idx}")(
            [attention_output, encoded_categorical_features]
        )
        # Layer normalization 1.
        x = layers.LayerNormalization(name=f"layer_norm1_{block_idx}", epsilon=1e-6)(x)
        # Feedforward.
        feedforward_output = create_mlp(
            hidden_units=[embedding_dims],
            dropout_rate=dropout_rate,
            activation=keras.activations.gelu,
            normalization_layer=layers.LayerNormalization(epsilon=1e-6),
            name=f"feedforward_{block_idx}",
        )(x)
        # Skip connection 2.
        x = layers.Add(name=f"skip_connection2_{block_idx}")([feedforward_output, x])
        # Layer normalization 2.
        encoded_categorical_features = layers.LayerNormalization(
            name=f"layer_norm2_{block_idx}", epsilon=1e-6
        )(x)

    # Flatten the "contextualized" embeddings of the categorical features.
    categorical_features = layers.Flatten()(encoded_categorical_features)
    # Apply layer normalization to the numerical features.
    numerical_features = layers.LayerNormalization(epsilon=1e-6)(numerical_features)
    # Prepare the input for the final MLP block.
    features = layers.concatenate([categorical_features, numerical_features])

    # Compute MLP hidden_units.
    mlp_hidden_units = [
        factor * features.shape[-1] for factor in mlp_hidden_units_factors
    ]
    # Create final MLP.
    features = create_mlp(
        hidden_units=mlp_hidden_units,
        dropout_rate=dropout_rate,
        activation=keras.activations.selu,
        normalization_layer=layers.BatchNormalization(),
        name="MLP",
    )(features)

    # Add a sigmoid as a binary classifer.
    outputs = layers.Dense(units=2, activation="softmax", name="softmax")(features) #change this 
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.save(path+'my_model.h5')
    return model


tabtransformer_model = create_tabtransformer_classifier(
    num_transformer_blocks=NUM_TRANSFORMER_BLOCKS,
    num_heads=NUM_HEADS,
    embedding_dims=EMBEDDING_DIMS,
    mlp_hidden_units_factors=MLP_HIDDEN_UNITS_FACTORS,
    dropout_rate=DROPOUT_RATE,
)

print("Total model weights:", tabtransformer_model.count_params())
# keras.utils.plot_model(tabtransformer_model, show_shapes=True, rankdir="LR")

PatientId
G.Nationality
Gender
C_Diabetes
C_HTN
C.Heart ischemic
C.Heart failure
C.Cardiomyopathies
C.End stage renal
C.Hemodialysis
C.COPD
C.Lung Interstitial Disease 
C.Bronchial Asthma
C.Cerebrovascular
C.Neurologic (dementia)
C.History of psychiatric
C.Cirrhosis
C.liver disease
C.Obesity
C.Sick cell
C.Cancer
C.Solid organ transplant
IS.Hematopoietic cell transplant
IS.HIV
IS.corticosteroids
IS.Other immunosuppressing agents
IS.Other immunodeficiencies
IS.Pregnancy
IS.Smoker
CSA_Fever
CSA_SOB
CSA.Chest pain
CSA.Confusion
CSA.Hemoptysis
CSA.Diarrhea
CSA_Cough
CSA.Myalgia
CSA.Headache
CSA.Abdominal Pain
CSA.Nausea or Vomiting
CSA.Loss of Smell or Tast
PO.Condition
PO.Current Condition
PEFF.Nasal Cannula
PEFF.Mask
PEFF.HFNC
PEFF.If patient need prone position
PEFF.If patient intubated
PEFF.If Patient required Psychiatric Consultation
PEFF.Presence of thrombo-embolic
PEFF.Confirmed DVT
PEFF.Confirmed Pulmonary embolism
PEFF.Confirmed Myocardial infarction
PEFF.Confirmed CVA/TIA
MPA_Anti

/home/o/oananbeh/.conda/envs/notebook-env/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Let's train and evaluate the TabTransformer model:

In [38]:
history = run_experiment(
    model=tabtransformer_model,
    train_data_file=train_data_file,
    test_data_file=test_data_file,
    num_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    batch_size=BATCH_SIZE,
)

Start training the model...
Epoch 1/15


/home/o/oananbeh/.conda/envs/notebook-env/lib/python3.9/site-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['INR', 'AST', 'CPK', 'Fibrinogen'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


7/7 [==============================] - 10s 383ms/step - loss: 3.2655 - accuracy: 0.4906 - categorical_true_positives: 26.0000 - get_f1: 0.6609 - precision: 0.5107 - recall: 1.0000 - val_loss: 2.5515 - val_accuracy: 0.6087 - val_categorical_true_positives: 14.0000 - val_get_f1: 0.6190 - val_precision: 0.4762 - val_recall: 1.0000
Epoch 2/15
7/7 [==============================] - 1s 86ms/step - loss: 5.1496 - accuracy: 0.7358 - categorical_true_positives: 39.0000 - get_f1: 0.6906 - precision: 0.5321 - recall: 1.0000 - val_loss: 9.6531 - val_accuracy: 0.5217 - val_categorical_true_positives: 12.0000 - val_get_f1: 0.6190 - val_precision: 0.4762 - val_recall: 1.0000
Epoch 3/15
7/7 [==============================] - 1s 88ms/step - loss: 6.4041 - accuracy: 0.7547 - categorical_true_positives: 40.0000 - get_f1: 0.6879 - precision: 0.5321 - recall: 1.0000 - val_loss: 23.7719 - val_accuracy: 0.4783 - val_categorical_true_positives: 11.0000 - val_get_f1: 0.6190 - val_precision: 0.4762 - val_recall

In [39]:
# model.summary()
